In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import yaml

In [ ]:
from primaite.simulator.network.networks import arcd_uc2_network


In [ ]:
net = arcd_uc2_network()

In [ ]:
random_node = list(net.nodes.keys())[0]
f = net.nodes[random_node].file_system.create_file(file_name="testfile")

In [ ]:
f.describe_state()

In [ ]:
def test_file_observation():
    from primaite.simulator.sim_container import Simulation
    from primaite.simulator.network.hardware.nodes.computer import Computer
    from primaite.game.actor.observations import FileObservation

    sim = Simulation()
    pc = Computer(hostname="beep", ip_address="123.123.123.123", subnet_mask="255.255.255.0")
    sim.network.add_node(pc)
    f = pc.file_system.create_file(file_name="dog.png")

    dog_file_obs = FileObservation(where=['network','nodes',pc.uuid,'file_system'])
    print(sim.describe_state())
test_file_observation()

In [ ]:
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.game.actor.observations import FileObservation, FolderObservation

sim = Simulation()
pc = Computer(hostname="beep", ip_address="123.123.123.123", subnet_mask="255.255.255.0")
sim.network.add_node(pc)
f = pc.file_system.create_file(file_name="dog.png")

state = sim.describe_state()

dog_file_obs = FileObservation(where=['network','nodes',pc.uuid,'file_system', 'folders','root','files','dog.png'])
root_folder_obs = FolderObservation(where=['network','nodes',pc.uuid,'file_system', 'folders','root'],files=[dog_file_obs])
print(dog_file_obs(state))
print(root_folder_obs(state))

In [ ]:
dog_file_obs.space

In [ ]:
root_folder_obs.space

In [ ]:
state

In [ ]:
import yaml

In [ ]:
with open('example_config.yaml', 'r') as file:
    conf = yaml.safe_load(file)

In [ ]:
conf['simulation']

In [44]:
import yaml
from primaite.simulator.sim_container import Simulation
from primaite.simulator.network.hardware.nodes.computer import Computer
from primaite.simulator.network.hardware.nodes.server import Server
from primaite.simulator.network.hardware.nodes.switch import Switch
from primaite.simulator.network.hardware.nodes.router import Router

from primaite.simulator.system.applications.database_client import DatabaseClient
from primaite.simulator.system.services.database_service import DatabaseService
from primaite.simulator.system.services.dns_client import DNSClient
from primaite.simulator.system.services.dns_server import DNSServer
from primaite.simulator.system.services.red_services.data_manipulation_bot import DataManipulationBot


In [42]:
# import yaml

from primaite.simulator.network.hardware.nodes.router import ACLAction
from primaite.simulator.network.transmission.network_layer import IPProtocol
from primaite.simulator.network.transmission.transport_layer import Port
from primaite.simulator.system.services.dns_server import DNSServer


class PrimaiteSession:

    def __init__(self):
        self.simulation: Simulation
        self.agents = []

    @classmethod
    def from_config(cls, cfg_path):
        ref_to_uuid = {}

        game = cls()
        with open(cfg_path, 'r') as file:
            conf = yaml.safe_load(file)
        
        #1. create nodes 
        sim = Simulation()
        net = sim.network
        nodes_cfg = conf['simulation']['network']['nodes']
        links_cfg = conf['simulation']['network']['links']
        for node_cfg in nodes_cfg:
            node_ref = node_cfg['ref']
            n_type = node_cfg['type']
            if n_type == 'computer':
                new_node = Computer(hostname = node_cfg['hostname'], 
                                    ip_address = node_cfg['ip_address'], 
                                    subnet_mask = node_cfg['subnet_mask'], 
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg['dns_server'])
            elif n_type == 'server':
                new_node = Server(hostname = node_cfg['hostname'], 
                                    ip_address = node_cfg['ip_address'], 
                                    subnet_mask = node_cfg['subnet_mask'], 
                                    default_gateway = node_cfg['default_gateway'],
                                    dns_server = node_cfg.get('dns_server'))
            elif n_type == 'switch':
                new_node = Switch(hostname = node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
            elif n_type == 'router':
                new_node = Router(hostname=node_cfg['hostname'],
                                  num_ports = node_cfg.get('num_ports'))
                if 'ports' in node_cfg:
                    for port_num, port_cfg in node_cfg['ports'].items():
                        new_node.configure_port(port=port_num, 
                                                ip_address=port_cfg['ip_address'],
                                                subnet_mask=port_cfg['subnet_mask'])
                if 'acl' in node_cfg:
                    for r_num, r_cfg in node_cfg['acl'].items():
                        # excuse the uncommon walrus operator ` := `. It's just here as a shorthand, so that we can do
                        # both of these things once: check if a key is defined, access and convert it to a 
                        # Port/IPProtocol. TODO Refactor
                        new_node.acl.add_rule(
                            action = ACLAction[r_cfg['action']],
                            src_port = None if not (p:=r_cfg.get('src_port')) else Port[p],
                            dst_port = None if not (p:=r_cfg.get('dst_port')) else Port[p],
                            protocol = None if not (p:=r_cfg.get('protocol')) else IPProtocol[p],
                            src_ip_address = r_cfg.get('ip_address'),
                            dst_ip_address = r_cfg.get('ip_address'),
                            position = r_num
                        )
            if 'services' in node_cfg:
                for service_cfg in node_cfg['services']:
                    service_ref = service_cfg['ref']
                    service_type = service_cfg['type']
                    service_types_mapping = {
                        'dns_server' : DNSServer,
                        'database_client': DatabaseClient,
                        'database_service': DatabaseService,
                        # 'database_backup': ,
                        'data_manipulation_bot': DataManipulationBot,
                        'dns_client': DNSClient,
                        # 'web_browser'
                    }
                    if service_type


            net.add_node(new_node)
            ref_to_uuid[node_ref] = new_node.uuid


        #2. start/setup simulation objects
        #3. create agents
        #4. set up agents' actions and observation spaces.
        game.simulation = sim
        return game

s = PrimaiteSession.from_config('example_config.yaml')
# print(s.simulation.describe_state())

2023-09-25 19:10:46,253: Added node 3d356af7-15f8-41d4-bb2a-423d5a2e5978 to Network 44950e57-81b0-4964-9b12-223592c785aa
2023-09-25 19:10:46,254::WARNING::primaite.simulator.network.container::181::Can't add node 3d356af7-15f8-41d4-bb2a-423d5a2e5978. It is already in the network.
2023-09-25 19:10:46,258: Added node 8a94447a-ccb3-47b2-b7ba-488e631f8246 to Network 44950e57-81b0-4964-9b12-223592c785aa
2023-09-25 19:10:46,262: Added node 4e3de72a-60ad-416f-abb7-da352a59d13b to Network 44950e57-81b0-4964-9b12-223592c785aa
2023-09-25 19:10:46,264: Added node 21ae4eed-489f-43a7-af0a-f70000714c79 to Network 44950e57-81b0-4964-9b12-223592c785aa
2023-09-25 19:10:46,267: Added node c4b7d388-c13e-4f4e-be61-5cf05989da71 to Network 44950e57-81b0-4964-9b12-223592c785aa
2023-09-25 19:10:46,270: Added node 12dc9e72-fb7c-4a8f-ab79-d7574863fa16 to Network 44950e57-81b0-4964-9b12-223592c785aa
2023-09-25 19:10:46,273: Added node 37ff728f-64b5-4203-8177-cf27f51dc7c9 to Network 44950e57-81b0-4964-9b12-223592

In [43]:
print(s.simulation.describe_state())

{'uuid': '532b534a-78b7-4630-92f9-626d775ee4e2', 'network': {'uuid': '44950e57-81b0-4964-9b12-223592c785aa', 'nodes': {'3d356af7-15f8-41d4-bb2a-423d5a2e5978': {'uuid': '3d356af7-15f8-41d4-bb2a-423d5a2e5978', 'hostname': 'router_1', 'operating_state': 2, 'NICs': {'c52a1516-3488-4011-95ae-44e77935720d': {'uuid': 'c52a1516-3488-4011-95ae-44e77935720d', 'ip_adress': '192.168.1.1', 'subnet_mask': '255.255.255.0', 'mac_address': '12:75:33:08:ae:04', 'speed': 100, 'mtu': 1500, 'wake_on_lan': False, 'enabled': False}, '075a3387-b3c5-4e30-b7d5-f39b82b1c9d8': {'uuid': '075a3387-b3c5-4e30-b7d5-f39b82b1c9d8', 'ip_adress': '192.168.1.1', 'subnet_mask': '255.255.255.0', 'mac_address': 'ce:67:79:4a:48:90', 'speed': 100, 'mtu': 1500, 'wake_on_lan': False, 'enabled': False}, '3fe76cb7-84fe-4592-b527-be69c06c8064': {'uuid': '3fe76cb7-84fe-4592-b527-be69c06c8064', 'ip_adress': '127.0.0.1', 'subnet_mask': '255.0.0.0', 'mac_address': '83:13:43:3a:e6:de', 'speed': 100, 'mtu': 1500, 'wake_on_lan': False, 'en